In [12]:
%%sh
cat > pi-mpi.c << EOF
#include <mpi.h>
#include <stdio.h>
#include <stdlib.h>

int main(int argc, char** argv) {
  int rank, size;

  MPI_Init(NULL, NULL);
  MPI_Comm_rank(MPI_COMM_WORLD, &rank);
  MPI_Comm_size(MPI_COMM_WORLD, &size);

  int start = rank * 10 + 1;
  int end = start + 9;
  int sum = 0;
  for (int i = start; i <= end; i++) {
    sum += i;
  }

  if (rank == 0) {
    int globalnasuma = sum;
    for (int i = 0; i < size - 1; i++) {
      MPI_Recv(&sum, 1, MPI_INT, i, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
      globalnasuma += sum;
    }
    double srednia = (double)globalnasuma / 100.0;
    printf("Globalna suma: %d\n", globalnasuma);
    printf("Globalna średnia: %f\n", srednia);
  }
  
  if (rank != 0) {
    MPI_Send(&sum, 1, MPI_INT, size - 1, 0, MPI_COMM_WORLD);
  } 

  MPI_Finalize();
  return 0;
}
EOF
mpicc pi-mpi.c && mpirun -n 1 --allow-run-as-root a.out

Globalna suma: 55
Globalna średnia: 0.550000
